In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Memory 처리 분석

In [ ]:
import os
from dotenv import load_dotenv
import argparse
import vertexai
from vertexai import agent_engines

from google.genai import types
from google.adk.runners import Runner

from google.adk.sessions import BaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import VertexAiSessionService

from google.adk.memory import BaseMemoryService
from google.adk.memory import InMemoryMemoryService 
from google.adk.memory import VertexAiMemoryBankService

from agent import search_agent, recall_agent

In [ ]:
# 환경 변수 정보.
from dotenv import dotenv_values
config = dotenv_values("../../.env")
for key, value in config.items():
    print(f"{key}: {value}")

#### Create Agent Engine for Memory Bank

In [ ]:
# Vertex AI 환경 초기화 : Agent Engine을 배포하기 위한 환경 설정
# Agent Engine 과 Gemini 모델을 사용하기 위한 리전을 다르게 할 수 있음.
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    # staging_bucket=os.getenv("AGENT_ENGINE_BUCKET"),
)


In [ ]:

agent_engine = agent_engines.create(display_name="memory_bank")

In [ ]:
agent_engine_id = agent_engine.resource_name.split("/")[-1]
print(agent_engine_id)

In [ ]:
APP_NAME = "AI_assistant"
USER_ID = "Forusone"
SESSION_ID = "forus_sess_001"

session_service = InMemorySessionService()

memory_service = VertexAiMemoryBankService(
    project  = os.environ['GOOGLE_CLOUD_PROJECT'],
    location = os.environ['GOOGLE_CLOUD_LOCATION'],
    agent_engine_id = agent_engine_id
)

#### 대화내용 메모리에 저장

In [ ]:
search_runner = Runner(
    agent=search_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

session = await search_runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,)

while True:

    user_input = input("\n 👤 User: ")
    
    if user_input.lower().strip() in ["exit", "quit", "bye"]:
                break
    
    content = types.Content(role='user', parts=[types.Part(text=user_input)])

    async for event in search_runner.run_async(user_id = session.user_id, 
                            session_id = session.id, 
                            new_message = content):
        
        if event.is_final_response():
            final_response_text = event.content.parts[0].text
            print(f"\n 🤖 AI Assistant: {final_response_text}")


completed_session = await search_runner.session_service.get_session(app_name=APP_NAME, 
                                                user_id=session.user_id, 
                                                session_id=session.id)

print("\n-- 검색 세션을 메모리에 추가 중 ---")
await memory_service.add_session_to_memory(completed_session)

print("\t 세션이 메모리에 추가되었습니다.")

#### 메모리에 있는 내용을 통한 대화.

In [ ]:
recall_runner = Runner(
    agent=recall_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,)

session = await recall_runner.session_service.create_session(
            app_name=APP_NAME, 
            user_id=USER_ID,)

while True:

    user_input = input("\n 👤 User: ")
    if user_input.strip().lower() in ["exit", "quit"]:
        break
    
    content = types.Content(role='user', parts=[types.Part(text=user_input)])

    async for event in recall_runner.run_async(user_id=session.user_id, 
                                session_id=session.id, 
                                new_message=content):

        if event.is_final_response():
            final_response_text = event.content.parts[0].text
            print(f"\n 🤖 AI Assistant: {final_response_text}")